GoodThought NGO focuses on education, healthcare, and sustainable development initiatives
to support underprivileged communities worldwide.

In this project, I explored the GoodThought PostgreSQL database, which contains records
of assignments, donations, and donors from 2010 to 2023. Using SQL, I analyzed how funding
and donor activity relate to project outcomes.

The database includes:
- **`Assignments`:** project details including duration, budget, region, and impact score
- **`Donations`:** records of financial contributions linked to donors and assignments
- **`Donors`:** information about individuals and organizations that fund projects

An entity relationship diagram (ERD) is included in the project documentation to illustrate table relationships.

In [1]:
/*
Query: highest_donation_assignments

Goal:
Identify the top five assignments by total donation value, broken out by donor type.

Output:
- assignment_name
- region
- rounded_total_donation_amount
- donor_type
*/

WITH assignment_donation_totals AS (
    SELECT
        d.assignment_id,
        donors.donor_type,
        ROUND(SUM(d.amount), 2) AS rounded_total_donation_amount
    FROM donations AS d
    JOIN donors
        ON d.donor_id = donors.donor_id
    GROUP BY
        d.assignment_id,
        donors.donor_type
)

SELECT
    a.assignment_name,
    a.region,
    adt.rounded_total_donation_amount,
    adt.donor_type
FROM assignment_donation_totals AS adt
JOIN assignments AS a
    ON adt.assignment_id = a.assignment_id
ORDER BY
    adt.rounded_total_donation_amount DESC
LIMIT 5;

,assignment_name,region,rounded_total_donation_amount,donor_type
0,Assignment_3033,East,3840.66,Individual
1,Assignment_300,West,3133.98,Organization
2,Assignment_4114,North,2778.57,Organization
3,Assignment_1765,West,2626.98,Organization
4,Assignment_268,East,2488.69,Individual


In [2]:
/*
Query: top_regional_impact_assignments

Goal:
For each region, return the single assignment with the highest impact_score,
including only assignments that have received at least one donation.

Output:
- assignment_name
- region
- impact_score
- num_total_donations
*/

WITH donated_assignments AS (
    SELECT
        a.assignment_id,
        a.assignment_name,
        a.region,
        a.impact_score,
        COUNT(*) AS num_total_donations
    FROM donations AS d
    JOIN assignments AS a
        ON d.assignment_id = a.assignment_id
    GROUP BY
        a.assignment_id,
        a.assignment_name,
        a.region,
        a.impact_score
),
ranked_by_region AS (
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY region
            ORDER BY impact_score DESC, assignment_id ASC
        ) AS impact_rank
    FROM donated_assignments
)

SELECT
    assignment_name,
    region,
    impact_score,
    num_total_donations
FROM ranked_by_region
WHERE impact_rank = 1
ORDER BY region ASC;

,assignment_name,region,impact_score,num_total_donations
0,Assignment_316,East,10.00,2
1,Assignment_2253,North,9.99,1
2,Assignment_3547,South,10.00,1
3,Assignment_2794,West,9.99,2


In [7]:
SELECT 'assignments' AS table_name, COUNT(*) AS row_count FROM assignments
UNION ALL
SELECT 'donations', COUNT(*) FROM donations
UNION ALL
SELECT 'donors', COUNT(*) FROM donors;

,donor_id,donor_name,donor_type
0,1,Donor_1,Individual
1,2,Donor_2,Organization
2,3,Donor_3,Individual
3,4,Donor_4,Organization
4,5,Donor_5,Organization
5,6,Donor_6,Corporate
6,7,Donor_7,Individual
7,8,Donor_8,Corporate
8,9,Donor_9,Individual
9,10,Donor_10,Individual
